# Availability Groups

***You will need to have followed the steps in the 00-CreateContainers notebook to use this notebook***

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

Setting up availabilty groups is hard

with dbatools it is a little easier :-)

The next block sets the variables for the instances and folder paths for this Notebook and checks the connection - Refer to the first notebook for any issues

In [42]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}
# Run this first to make sure output width does not mess with output - Update output buffer size to prevent clipping in Visual Studio output window.
if( $Host -and $Host.UI -and $Host.UI.RawUI ) {
    $rawUI = $Host.UI.RawUI
    $oldSize = $rawUI.BufferSize
    $typeName = $oldSize.GetType( ).FullName
    $newSize = New-Object $typeName (500, $oldSize.Height)
    $rawUI.BufferSize = $newSize
  }

 Creating connection to the containers


We have a connection to the containers


**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason


In [43]:
Get-DbaDatabase -SqlInstance $SQL2 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false

ComputerName : localhost


InstanceName : MSSQLSERVER


SqlInstance  : beardsql02


Database     : pubs


Status       : Dropped


...

ComputerName : localhost


InstanceName : MSSQLSERVER


SqlInstance  : beardsql02


Database     : Northwind


Status       : Dropped


# Containers

dbatools is not able to create availability groups in containers, so we will create an availability group below using T-SQL. If you are not using containers you can create an availability group with dbatools like this

````
$params = @{
    Primary = $SqlInstances[0]
    PrimarySqlCredential =  $SqlCredential 
    Secondary =   $SqlInstances[1]
    SecondarySqlCredential = $SqlCredential 
    Name = 'TheBeard_AG'
    Database = 'pubs','NorthWind','AdventureWorks2017'
    ClusterType = "None"
    SeedingMode = "Automatic"
    FailoverMode = "Manual"
    Confirm = $false
 }
 
# execute the command
 New-DbaAvailabilityGroup @params
 ````
 
There are many options that you can use here. Read the docs or get our book [beard.media\book]([beard.media\book)

Lets create an availability group to use with this notebook

First we shall create a master key on each instance

In [44]:
New-DbaDbMasterKey -SqlInstance $SQL1 -Credential $SqlCredential -Confirm:$false
New-DbaDbMasterKey -SqlInstance $SQL2 -Credential $SqlCredential -Confirm:$false

ComputerName        : localhost


InstanceName        : MSSQLSERVER


SqlInstance         : beardsql01


Database            : master


CreateDate          : 06/08/2021 11:14:29


...

SqlInstance         : beardsql02


Database            : master


CreateDate          : 06/08/2021 11:14:29


DateLastModified    : 06/08/2021 11:14:29


IsEncryptedByServer : True


# Certificate

Next we will create a certificate for the endpoints on SQL1, back it up and restore it on SQL2. You can do this for any certificate that you require by the way :-)

In [45]:
New-DbaDbCertificate -SqlInstance $sql1 -Name hadr_cert -Subject hadr_cert -Confirm:$false

$EncryptionPassword = ConvertTo-SecureString -AsPlainText "Password4567!!" -force
Backup-DbaDbCertificate -SqlInstance $SQL1 -Suffix $null -Certificate hadr_cert -Path '/var/opt/mssql/backups/' -EncryptionPassword $EncryptionPassword -Confirm:$false


ComputerName                 : localhost


InstanceName                 : MSSQLSERVER


SqlInstance                  : beardsql01


Database                     : master


Name                         : hadr_cert


...

InstanceName : MSSQLSERVER


Key          : /var/opt/mssql/backups/hadr_cert.pvk


Path         : /var/opt/mssql/backups/hadr_cert.cer


SqlInstance  : beardsql01


Status       : Success


Now we will use `Get-DbaFile` to ge the certificate path and then restore the certificate to SQL2
This will prompt you to ask for input. If you did not wnat this, you can use `-Force` I am also showing verbose output here. All dbatools commands can output verbose information if you use the `-Verbose` switch

In [46]:
$Certificate = (Get-DbaFile -SqlInstance $sql2 -Path '/var/opt/mssql/backups/' -FileType cer).FileName
Restore-DbaDbCertificate -SqlInstance $SQL2 -Path $Certificate -DecryptionPassword $EncryptionPassword  -Verbose -Confirm:$false


VERBOSE: [12:14:38][Restore-DbaDbCertificate] Processing /var/opt/mssql/backups/\hadr_cert.cer


VERBOSE: Performing the operation "Importing Certificate" on target "hadr_cert on localhost,15593".


VERBOSE: [12:14:38][Restore-DbaDbCertificate] Creating Certificate: hadr_cert


VERBOSE: [12:14:38][Restore-DbaDbCertificate] Full certificate path: \var\opt\mssql\backups\\hadr_cert.cer


VERBOSE: [12:14:38][Restore-DbaDbCertificate] Private key: \var\opt\mssql\backups\\hadr_cert.pvk


...

Issuer                       : hadr_cert


LastBackupDate               : 01/01/0001 00:00:00


Owner                        : dbo


PrivateKeyEncryptionType     : MasterKey


Serial                       : b5 1f 18 4d e7 c2 ca 07


# Endpoints
Now that we have the certificates on each instance, we can create the endpoints using the certificate and start them

In [47]:
New-DbaEndpoint -SqlInstance $sql1 -Name hadr_endpoint -Port 5022 -Certificate hadr_cert 
New-DbaEndpoint -SqlInstance $sql2 -Name hadr_endpoint -Port 5023 -Certificate hadr_cert

Start-DbaEndpoint -SqlInstance $SQL1 -EndPoint hadr_endpoint
Start-DbaEndpoint -SqlInstance $SQL2 -EndPoint hadr_endpoint

ComputerName    : localhost


InstanceName    : MSSQLSERVER


SqlInstance     : beardsql01


ID              : 65536


Name            : hadr_endpoint


...

EndpointType    : DatabaseMirroring


Owner           : sqladmin


IsAdminEndpoint : False


Fqdn            : TCP://f443490967e7:5023


IsSystemObject  : False


# Create the Availability Group

we have to use T-SQL to get the right information only because we are using containers

In [48]:

$query = @"
CREATE AVAILABILITY GROUP [TheBeard_AG]
    WITH (CLUSTER_TYPE = NONE)
    FOR REPLICA ON
        N'beardsql01' WITH (
            ENDPOINT_URL = N'tcp://beardsql01:5022',
		    AVAILABILITY_MODE = ASYNCHRONOUS_COMMIT,
		    FAILOVER_MODE = MANUAL,
		    SEEDING_MODE = AUTOMATIC,
                    SECONDARY_ROLE (ALLOW_CONNECTIONS = ALL)
		    ),
        N'beardsql02' WITH ( 
		    ENDPOINT_URL = N'tcp://beardsql02:5023', 
		    AVAILABILITY_MODE = ASYNCHRONOUS_COMMIT,
		    FAILOVER_MODE = MANUAL,
		    SEEDING_MODE = AUTOMATIC,
		    SECONDARY_ROLE (ALLOW_CONNECTIONS = ALL)
		    );
		
ALTER AVAILABILITY GROUP [TheBeard_AG] GRANT CREATE ANY DATABASE;
"@

Invoke-DbaQuery -SqlInstance $sql1 -Query $query
Write-Output "Created AG"
$query = @"
ALTER AVAILABILITY GROUP [TheBeard_AG] JOIN WITH (CLUSTER_TYPE = NONE);
		 
ALTER AVAILABILITY GROUP [TheBeard_AG] GRANT CREATE ANY DATABASE;
"@

Invoke-DbaQuery -SqlInstance $sql2 -Query $query
Write-Output "Added SQL2 to the AG"

Created AG


Added SQL2 to the AG


we can check that the availability groups on an instance using `Get-DbaAvailabilityGroup`

In [49]:
Get-DbaAvailabilityGroup -SqlInstance $sql1 

ComputerName               : localhost


InstanceName               : MSSQLSERVER


SqlInstance                : beardsql01


LocalReplicaRole           : Primary


AvailabilityGroup          : TheBeard_AG


...

DtcSupportEnabled          : False


AutomatedBackupPreference  : Secondary


AvailabilityReplicas       : {beardsql01, beardsql02}


AvailabilityDatabases      : {}


AvailabilityGroupListeners : {}


Excellent, we have an availablity group - Lets check the replicas

In [50]:
Get-DbaAgReplica -SqlInstance $sql1

ComputerName               : localhost


InstanceName               : MSSQLSERVER


SqlInstance                : beardsql01


AvailabilityGroup          : TheBeard_AG


Name                       : beardsql01


...

BackupPriority             : 50


EndpointUrl                : tcp://beardsql02:5023


SessionTimeout             : 10


FailoverMode               : Manual


ReadonlyRoutingList        : {}


We have two replicas - Right now they are not synchronising because there are no databases, You can use this command to see the rol up status for the replicas

Let's check the databases

In [51]:
Get-DbaAgDatabase -SqlInstance $SQL1

There are no databases - Lets add the databases

## Compatability Level

In [52]:
Get-DbaDbCompatibility -SqlInstance $Sql1,$sql2 | ft

ComputerName InstanceName SqlInstance Database           Compatibility


------------ ------------ ----------- --------           -------------


localhost    MSSQLSERVER  beardsql01  master                Version150


localhost    MSSQLSERVER  beardsql01  tempdb                Version150


localhost    MSSQLSERVER  beardsql01  model                 Version150


...

localhost    MSSQLSERVER  beardsql01  Northwind             Version150


localhost    MSSQLSERVER  beardsql02  master                Version150


localhost    MSSQLSERVER  beardsql02  tempdb                Version150


localhost    MSSQLSERVER  beardsql02  model                 Version150


localhost    MSSQLSERVER  beardsql02  msdb                  Version150


# Recovery Mode

We need ot set the recovery mode of the databases to full

In [53]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeSystem | Set-DbaDbRecoveryModel -RecoveryModel Full -Confirm:$false

ComputerName   : localhost


InstanceName   : MSSQLSERVER


SqlInstance    : beardsql01


Name           : pubs


Status         : Normal


...

IsAccessible   : True


RecoveryModel  : Full


LastFullBackup : 21/12/2019 14:02:11


LastDiffBackup : 01/01/0001 00:00:00


LastLogBackup  : 01/01/0001 00:00:00


# Backup

Of course, having set the recovery model to full we need to back up the databases otherwise they are in PSUDOSIMPLE mode. You can see this. If we check the recovery model, it will show full

In [54]:
Get-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs | Format-Table

ComputerName InstanceName SqlInstance Name               Status IsAccessible RecoveryModel LastFullBackup      LastDiffBackup      LastLogBackup


------------ ------------ ----------- ----               ------ ------------ ------------- --------------      --------------      -------------


localhost    MSSQLSERVER  beardsql01  pubs               Normal         True          Full 21/12/2019 14:02:00 01/01/0001 00:00:00 01/01/0001 00:00:00


localhost    MSSQLSERVER  beardsql01  AdventureWorks2017 Normal         True          Full 27/10/2017 14:35:39 01/01/0001 00:00:00 01/01/0001 00:00:00


localhost    MSSQLSERVER  beardsql01  Northwind          Normal         True          Full 21/12/2019 14:02:11 01/01/0001 00:00:00 01/01/0001 00:00:00


but if we use `Test-DbaDbRecoveryModel` you can see that the *Actual* recovery model is SIMPLE

In [55]:
Test-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs | Format-Table

ComputerName InstanceName SqlInstance Database           ConfiguredRecoveryModel ActualRecoveryModel


------------ ------------ ----------- --------           ----------------------- -------------------


f443490967e7 MSSQLSERVER  beardsql01  pubs               FULL                    SIMPLE


f443490967e7 MSSQLSERVER  beardsql01  AdventureWorks2017 FULL                    SIMPLE


f443490967e7 MSSQLSERVER  beardsql01  Northwind          FULL                    SIMPLE


If we backup the databases (like we did in the BackupsandRestores notebook

In [56]:
Backup-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 -Database  AdventureWorks2017, Northwind, pubs -Type Full -IgnoreFileChecks
Backup-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 -Database  AdventureWorks2017, Northwind, pubs -Type Log -IgnoreFileChecks

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 Backing up database 1 of 3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 Backing up database 1 of 3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 Backing up database 1 of 3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 Backing up database 1 of 3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 Backing up database 1 of 3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

...

beardsql01  Northwind          Full 6.84 MB   Disk       2021-08-06 11:15:29.000 00:00:01 2021-08-06 11:15:30.000


beardsql01  pubs               Full 4.59 MB   Disk       2021-08-06 11:15:30.000 00:00:00 2021-08-06 11:15:30.000


beardsql01  AdventureWorks2017 Log  84.00 KB  Disk       2021-08-06 11:15:30.000 00:00:00 2021-08-06 11:15:30.000


beardsql01  Northwind          Log  80.00 KB  Disk       2021-08-06 11:15:31.000 00:00:00 2021-08-06 11:15:31.000


beardsql01  pubs               Log  80.00 KB  Disk       2021-08-06 11:15:31.000 00:00:00 2021-08-06 11:15:31.000


and test our recovery model again

In [57]:
Test-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs | Format-Table

ComputerName InstanceName SqlInstance Database           ConfiguredRecoveryModel ActualRecoveryModel


------------ ------------ ----------- --------           ----------------------- -------------------


f443490967e7 MSSQLSERVER  beardsql01  pubs               FULL                    FULL


f443490967e7 MSSQLSERVER  beardsql01  AdventureWorks2017 FULL                    FULL


f443490967e7 MSSQLSERVER  beardsql01  Northwind          FULL                    FULL


All our databases are now in FULL recovery :-)

Let's add them to the Availability Group

In [58]:
$AddAgDbConfig = @{
    SqlInstance = $SqlInstances[0] 
    SqlCredential = $SqlCredential 
    AvailabilityGroup = 'TheBeard_AG' 
    Database = 'Northwind','Pubs','AdventureWorks2017' 
    Secondary = $SqlInstances[01] 
    SecondarySqlCredential = $SqlCredential 
    SeedingMode = 'Automatic' 
}
Add-DbaAgDatabase @AddAgDbConfig

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 Adding database(s) to Availability Group TheBeard_AG.                                                                                                                                                                                                                                                                                                                                                                                                                                                             

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 Adding database(s) to Availability Group TheBeard_AG.                                                                                                                                                                                                                                                                                                                                                                                                                                                             

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 Adding database(s) to Availability Group TheBeard_AG.                                                                                                                                                                                                                                                                                                                                                                                                                                                             

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 Adding database Northwind to Availability Group TheBeard_AG.                                                                                                                                                                                                                                                                                                                                                                                                                                                      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 Adding database Northwind to Availability Group TheBeard_AG.                                                                                                                                                                                                                                                                                                                                                                                                                                                      

...

Name                 : AdventureWorks2017


SynchronizationState : Synchronizing


IsFailoverReady      : False


IsJoined             : True


IsSuspended          : False


Let's have a look at them


In [60]:
$sql1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential
$sql2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
Get-DbaAgDatabase -SqlInstance $SQL1 | Format-Table

ComputerName InstanceName SqlInstance AvailabilityGroup LocalReplicaRole Name               SynchronizationState IsFailoverReady IsJoined IsSuspended


------------ ------------ ----------- ----------------- ---------------- ----               -------------------- --------------- -------- -----------


localhost    MSSQLSERVER  beardsql01  TheBeard_AG                Primary AdventureWorks2017         Synchronized            True     True       False


localhost    MSSQLSERVER  beardsql01  TheBeard_AG                Primary Northwind                  Synchronized            True     True       False


localhost    MSSQLSERVER  beardsql01  TheBeard_AG                Primary Pubs                       Synchronized            True     True       False


You can look in SSMS if you like :-)

![AG](.\images\ag.png )

## Suspending and resuming data movement

You can use dbatools to suspend and resume data movement for your Availability Group databases. The code below will prompt you to confirm that you wish to suspend them.

In [61]:
Get-DbaAgDatabase -SqlInstance $sql1  | Suspend-DbaAgDbDataMovement

Confirm


Are you sure you want to perform this action?
Performing the operation "Seting availability group  to " on target "".


[Y] Yes  

[A] Yes to All  

[N] No  

...

Name                 : Pubs


SynchronizationState : NotSynchronizing


IsFailoverReady      : True


IsJoined             : True


IsSuspended          : True


You can check that they are suspended using `Get-DbaAgDatabase` You will see that the `SynchronizationState` has changed to not synchronising and the `IsSuspended` property is set to true

In [68]:
Get-DbaAgDatabase -SqlInstance $sql1 |Format-Table

ComputerName InstanceName SqlInstance AvailabilityGroup LocalReplicaRole Name               SynchronizationState IsFailoverReady IsJoined IsSuspended


------------ ------------ ----------- ----------------- ---------------- ----               -------------------- --------------- -------- -----------


localhost    MSSQLSERVER  beardsql01  TheBeard_AG                Primary AdventureWorks2017         Synchronized            True     True       False


localhost    MSSQLSERVER  beardsql01  TheBeard_AG                Primary Northwind                  Synchronized            True     True       False


localhost    MSSQLSERVER  beardsql01  TheBeard_AG                Primary Pubs                       Synchronized            True     True       False


If you want to see if any databases in your Availability Group are not synchronised you can use the code below (if you are looking on an asynchronous replica then you would change Synchronized to Synchronizing.

In [67]:
Get-DbaAgDatabase -SqlInstance $SQL1 | Where SynchronizationState -ne 'Synchronized' | Format-Table

You can add the capability to resume data movement using dbatools as well with `Resume-DbaAgDbDataMovement` this time we will use `-Confirm:$false` so there are no prompts

In [65]:
Get-DbaAgDatabase -SqlInstance $SQL1| Resume-DbaAgDbDataMovement -Confirm:$false

ComputerName         : localhost


InstanceName         : MSSQLSERVER


SqlInstance          : beardsql01


AvailabilityGroup    : TheBeard_AG


LocalReplicaRole     : Primary


...

Name                 : Pubs


SynchronizationState : Synchronized


IsFailoverReady      : True


IsJoined             : True


IsSuspended          : False


## Removing a database from an Availability Group

You can also remove a database from the availablity group. This code will prompt you to confirm the changing action which is usual dbatools behaviour


In [69]:
Remove-DbaAgDatabase -SqlInstance $SQL1 -AvailabilityGroup TheBeard_AG -Database pubs

Confirm


Are you sure you want to perform this action?
Performing the operation "Removing availability group database [Pubs]" on target "localhost,15592".


[Y] Yes  

[A] Yes to All  

[N] No  

...

InstanceName      : MSSQLSERVER


SqlInstance       : beardsql01


AvailabilityGroup : TheBeard_AG


Database          : Pubs


Status            : Removed


We can check the user databases on the instances with `Get-DbaDatabase`

In [71]:
$sql1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential
$sql2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
Get-DbaDatabase -SqlInstance $SQL1, $SQL2 -ExcludeSystem | Select SqlInstance, Name,Status

SqlInstance Name                  Status


----------- ----                  ------


beardsql01  pubs                  Normal


beardsql01  AdventureWorks2017    Normal


beardsql01  Northwind             Normal


beardsql02  Northwind             Normal


beardsql02  Pubs               Restoring


beardsql02  AdventureWorks2017    Normal


You can see that the pubs database on the secondary is in restoring state.

You can also use `Get-DbaDatabase` to see the Availability Group synchronisation state and the Availability Group that it is joined to by getting some of the properties that are not displayed by default

In [72]:
Get-DbaDatabase -SqlInstance $SQL1, $SQL2 -ExcludeSystem | Select SqlInstance, Name, AvailabilityDatabaseSynchronizationState, AvailabilityGroupName 

SqlInstance Name               AvailabilityDatabaseSynchronizationState AvailabilityGroupName


----------- ----               ---------------------------------------- ---------------------


beardsql01  pubs                                                        


beardsql01  AdventureWorks2017 Synchronized                             TheBeard_AG


beardsql01  Northwind          Synchronized                             TheBeard_AG


beardsql02  Northwind          Synchronizing                            TheBeard_AG


beardsql02  Pubs                                                        


beardsql02  AdventureWorks2017 Synchronizing                            TheBeard_AG


You can see that the pubs database is not joined to the Availability Group

# Failover

You can use dbatools to failover the Availability Group. Lets take a look at the Availability Group

In [73]:
Get-DbaAvailabilityGroup -SqlInstance $SQL1

ComputerName               : localhost


InstanceName               : MSSQLSERVER


SqlInstance                : beardsql01


LocalReplicaRole           : Primary


AvailabilityGroup          : TheBeard_AG


...

DtcSupportEnabled          : False


AutomatedBackupPreference  : Secondary


AvailabilityReplicas       : {beardsql01, beardsql02}


AvailabilityDatabases      : {AdventureWorks2017, Northwind}


AvailabilityGroupListeners : {}


The primary replica for the Availability Group is SQL1. Lets fail it over to SQL2

In [74]:
 Invoke-DbaAgFailover -SqlInstance $SQL2 -AvailabilityGroup TheBeard_AG

Confirm


Are you sure you want to perform this action?
Performing the operation "Gracefully failing over TheBeard_AG" on target "localhost,15593".


[Y] Yes  

[A] Yes to All  

[N] No  

[L] No to All  

[S] Suspend  

[?] Help

(default is 'Y')

It has prompted us for confirmation and then told us that the cluster type that we have used for the containers does not support this type of failover so we need to use the force                     parameter

In [75]:
 Invoke-DbaAgFailover -SqlInstance $SQL2 -AvailabilityGroup TheBeard_AG -Force

ComputerName               : localhost


InstanceName               : MSSQLSERVER


SqlInstance                : beardsql02


LocalReplicaRole           : Primary


AvailabilityGroup          : TheBeard_AG


...

DtcSupportEnabled          : False


AutomatedBackupPreference  : Secondary


AvailabilityReplicas       : {beardsql01, beardsql02}


AvailabilityDatabases      : {AdventureWorks2017, Northwind}


AvailabilityGroupListeners : {}


When we check the Availability Group on SQL1 again

In [78]:
$sql1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential
$sql2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
$SqlInstances = $sql1,$sql2
Get-DbaAvailabilityGroup -SqlInstance $SQL1

ComputerName               : localhost


InstanceName               : MSSQLSERVER


SqlInstance                : beardsql01


LocalReplicaRole           : Secondary


AvailabilityGroup          : TheBeard_AG


...

DtcSupportEnabled          : False


AutomatedBackupPreference  : Secondary


AvailabilityReplicas       : {beardsql01, beardsql02}


AvailabilityDatabases      : {AdventureWorks2017, Northwind}


AvailabilityGroupListeners : {}


You can see by comparing the output that the primary replica has swapped to the other container.

In [79]:
Get-DbaDatabase -SqlInstance $SqlInstances -SqlCredential $SqlCredential -ExcludeSystem | Select SqlInstance, Name, AvailabilityDatabaseSynchronizationState, AvailabilityGroupName 

SqlInstance Name               AvailabilityDatabaseSynchronizationState AvailabilityGroupName


----------- ----               ---------------------------------------- ---------------------


beardsql01  pubs                                                        


beardsql01  AdventureWorks2017 NotSynchronizing                         TheBeard_AG


beardsql01  Northwind          NotSynchronizing                         TheBeard_AG


beardsql02  Northwind          Synchronized                             TheBeard_AG


beardsql02  Pubs                                                        


beardsql02  AdventureWorks2017 Synchronized                             TheBeard_AG


# Clean Up

The 99-CleanUpContainers notebook will remove the containers, files and directory - it will leave the image so you do not have to download it again!